In [ ]:
# pip install pmdarima

In [ ]:
import pandas as pd
import numpy as np
from pmdarima import auto_arima
from sklearn.metrics import root_mean_squared_error


In [ ]:
df = pd.read_excel('../Input/DadosCompeticao.xlsx')

#### Autoarima -> WRMSE = 0.07500016217479046
    ** Usando sazonalidade

In [ ]:
forecast_window = 12

In [ ]:
def get_forecast(df, forecast_window)-> (set, set):
    forecasts = {}
    scores = {}

    for col in df.columns:
        print(f'\n ==== Treinamento para o modelo {col} ====')
        series = df[col]

        try:
            if len(series) > forecast_window:
                train = series[:-forecast_window]
                test = series[-forecast_window:]
            else:
                train = series
                test = None

            model = auto_arima(train,
                               seasonal=True,
                               stepwise=True,
                               suppress_warnings=True,
                               error_action='ignore')

            forecast = model.predict(n_periods=forecast_window)
            forecasts[col] = forecast

            if test is not None and len(test) == forecast_window:
                rmse = root_mean_squared_error(test, forecast)
                scores[col] = rmse
            else:
                scores[col] = np.nan

        except Exception as e:
            print(f"Erro em {col}: {e}")
            forecasts[col] = np.full(forecast_window, np.nan)
            scores[col] = np.nan

    return forecasts, scores

In [ ]:
forecast, score = get_forecast(df, forecast_window)

In [ ]:
df_forecasts = pd.DataFrame(forecast)
df_scores = pd.DataFrame(list(score.items()), columns=['Série', 'RMSE'])

In [ ]:
df_forecasts.to_excel('../Output/v3/previsoes_arima.xlsx', index=False)
df_scores.to_excel('../Output/v3/scores_arima.xlsx')

In [ ]:
wrmse = lambda rmse: (rmse[:11] * (1 / 11)).sum()

In [20]:
print(f'WRMSE = {wrmse(df_scores["RMSE"])}')

WRMSE = 0.07500016217479046
